In [1]:

import csv
import numpy as np
import pandas as pd
import os
from textblob import TextBlob


Sentiment(polarity=0.16666666666666669, subjectivity=0.5166666666666666)


In [2]:
X_train = pd.read_csv("/code/data/train.csv")['headline']
y_train = pd.read_csv("/code/data/train.csv")['is_sarcastic']

X_val = pd.read_csv("/code/data/val.csv")['headline']
y_val = pd.read_csv("/code/data/val.csv")['is_sarcastic']

In [3]:
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import TfidfVectorizer

my_stop_words = text.ENGLISH_STOP_WORDS.union([
    "of", "the", "in", "for", "a", "on", "with", "and", "is",
    "from", "at", "about", "by"
])

vectorizer = TfidfVectorizer(ngram_range=(1,1), stop_words=my_stop_words)

# Initialize the `tfidf_vectorizer` 
#tfidf_vectorizer = TfidfVectorizer(stop_words='english') 
tfidf_vectorizer = TfidfVectorizer()

# Fit and transform the training data 
tfidf_train = tfidf_vectorizer.fit_transform(X_train) 

# Transform the val set 
tfidf_val = tfidf_vectorizer.transform(X_val)

## Sentiment analysis 

In [37]:
polarity_scores_train = []
subjectivity_scores_train = []
for row in X_train:
# sentiment = TextBlob(tweets.to_string())
    sentiment = TextBlob(row)
    polarity_scores_train.append(sentiment.sentiment.polarity)
    subjectivity_scores_train.append(sentiment.sentiment.subjectivity)

In [36]:
polarity_scores_val = []
subjectivity_scores_val = []
for row in X_val:
# sentiment = TextBlob(tweets.to_string())
    sentiment = TextBlob(row)
    polarity_scores_val.append(sentiment.sentiment.polarity)
    subjectivity_scores_val.append(sentiment.sentiment.subjectivity)

In [38]:
polarity_train = pd.DataFrame()
for i, polarity_score_train in enumerate(polarity_scores_train):
    polarity_train[i] = polarity_score_train

In [39]:
polarity_val = pd.DataFrame()
for i, polarity_score_val in enumerate(polarity_scores_val):
    polarity_val[i] = polarity_score_val

In [40]:
polarity_train = pd.DataFrame()
polarity_train['polarity'] = polarity_scores_train

In [42]:
polarity_val = pd.DataFrame()
polarity_val['polarity'] = polarity_scores_val

In [66]:
from scipy.sparse import hstack
features_train = hstack((tfidf_train,np.array(polarity_scores_train)[:,None])).A
features_val = hstack((tfidf_val,np.array(polarity_scores_val)[:,None])).A

In [63]:
features.shape

(17171, 20930)

In [65]:
tfidf_train

<17171x20929 sparse matrix of type '<class 'numpy.float64'>'
	with 169396 stored elements in Compressed Sparse Row format>

In [67]:
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics

clf = MultinomialNB() 

# Predict bag of words

clf.fit(features_train, y_train)
pred = clf.predict(features_val)
metrics.accuracy_score(y_val, pred)

ValueError: Input X must be non-negative